In [1]:
import numpy as np
from ShillingAttack import Attack
import random 
import math
import csv
from ML100KPrePreocessing import ML100KRatingMatrix
from AmazonSoftwarePreprocessing import AmazonSoftwareRatingMatrix
from AmazonFashionPreProcessing import AmazonFashionRatingMatrix

In [2]:
class BandwagonAttack(Attack):
    def __init__(self,attacksize,fillersize,targetItems,k,dataset):
        super().__init__(attacksize,fillersize,targetItems,k,dataset)
    
    def rateSelectedItems(self):
        sumOfRatings = np.sum(self.ratingMatrix, axis = 0)
        self.popularItems = np.argsort(sumOfRatings)[::-1]
        
        for n in range(self.selectedItemSetSize):
            self.selectedItems[n] = self.popularItems[n]
            
        for i in range(self.attackSize):
            for j in self.selectedItems:
                j=int(j)
                self.shillingProfiles[i][j] = 5
                
    def createProfiles(self):
        self.pushItem() #use self.nukeItem case of a nuke attack 
        self.rateSelectedItems()
        self.fillerItems = self.randomNumberGenerator()
        
        mean = np.zeros(self.numberOfItems)
        #mean is an array of length 1682, where each element,i, is the mean of the i-th item
        sd = np.zeros(self.numberOfItems)
        #sd is an array of length 1682, with each element,i, being the sd of the i-th item

        for k in range(self.numberOfItems):
            nonZeroColumn = np.array(self.ratingMatrix[:,k])
            nonZeroColumn = nonZeroColumn[nonZeroColumn != 0]
            mean[k] = np.mean(nonZeroColumn)
            sd[k] = np.std(nonZeroColumn)
    
        for i in range (self.attackSize):
            j = 0
            while j < self.fillerSize:
                item = random.choice(self.fillerItems)
                if item not in self.shillingProfiles[i] and item not in self.selectedItems:
                    
                    max = mean[item - 1] + 3*sd[item - 1]
                    min = mean[item - 1] - 3*sd[item - 1]
                    
                    rating = random.gauss(mean[item - 1],sd[item - 1])
                    rating = math.ceil(1 + ((rating - min)/(max - min)) * 4) if max - min != 0 else 1
                    #normalizing the output of the gaussian to be between 1 and 5
                    
                    if rating > 5:
                        rating = 5
                    if rating < 1:
                        rating = 1
                        
                    self.shillingProfiles[i][item - 1] = rating
                    j += 1  
                    
    def mountAttack(self):
        self.createProfiles()
        self.injectProfiles()
        
        if isinstance(self.RM,ML100KRatingMatrix): 
        
            with open('ML100K_BandwagonAttack.csv','w') as file:
                docWriter = csv.writer(file, delimiter = ',')
                docWriter.writerows(self.ratingMatrix)
                
        elif isinstance(self.RM,AmazonFashionRatingMatrix):
        
            with open('AmazonFashion_BandwagonAttack.csv','w') as file:
                docWriter = csv.writer(file, delimiter = ',')
                docWriter.writerows(self.ratingMatrix)
        else: 
            with open('AmazonSoftware_BandwagonAttack.csv','w') as file:
                docWriter = csv.writer(file, delimiter = ',')
                docWriter.writerows(self.ratingMatrix)

In [3]:
def test ():
    Test = BandwagonAttack(95,169,[0,1,2],10,0)#change last input to change dataset
    shillingAttackMatrix = Test.mountAttack()

In [4]:
test()